In [2]:
import sys
sys.path.append('../')

In [3]:
# forcing the notebook to reload the modules
%load_ext autoreload
%autoreload 2

In [4]:
from src.data import datacollector as dc
from src.data import datafilter as df
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
import requests
from concurrent.futures import ThreadPoolExecutor


In [7]:
dc.collectfonts("../data/raw/fonts-main", "../data/processed/all-fonts")

Copy: ../data/raw/fonts-main\apache\aclonica\Aclonica-Regular.ttf -> ../data/processed/all-fonts\Aclonica-Regular.ttf
Copy: ../data/raw/fonts-main\apache\arimo\Arimo-Italic[wght].ttf -> ../data/processed/all-fonts\Arimo-Italic[wght].ttf
Copy: ../data/raw/fonts-main\apache\arimo\Arimo[wght].ttf -> ../data/processed/all-fonts\Arimo[wght].ttf
Copy: ../data/raw/fonts-main\apache\calligraffitti\Calligraffitti-Regular.ttf -> ../data/processed/all-fonts\Calligraffitti-Regular.ttf
Copy: ../data/raw/fonts-main\apache\cherrycreamsoda\CherryCreamSoda-Regular.ttf -> ../data/processed/all-fonts\CherryCreamSoda-Regular.ttf
Copy: ../data/raw/fonts-main\apache\chewy\Chewy-Regular.ttf -> ../data/processed/all-fonts\Chewy-Regular.ttf
Copy: ../data/raw/fonts-main\apache\comingsoon\ComingSoon-Regular.ttf -> ../data/processed/all-fonts\ComingSoon-Regular.ttf
Copy: ../data/raw/fonts-main\apache\cousine\Cousine-Bold.ttf -> ../data/processed/all-fonts\Cousine-Bold.ttf
Copy: ../data/raw/fonts-main\apache\cousi

In [4]:
df.filter_fonts("../data/processed/all-fonts", "../data/processed/filtered-fonts")

734it [00:10, 89.62it/s] 'created' timestamp seems very low; regarding as unix timestamp
3331it [00:51, 65.06it/s] 

Processed 3330 Fonts. Found 360 and moved them to ../data/processed/filtered-fonts.


This is not enough data to train a model. Seems like we need to download more data.

In [1]:
def download_fonts(url_list, download_dir=None):
    if download_dir is None:
        download_dir = os.getcwd()

    # if url_list is a list
    if isinstance(url_list, list):
        files_downloaded = 0
        for url in tqdm(url_list):
            try:
                response = requests.get(url)
                if response.status_code == 200:
                    font_file_name = os.path.basename(url)
                    font_file_name_path = os.path.join(download_dir, font_file_name)
                    with open(font_file_name_path, 'wb') as f:
                        f.write(response.content)
                        files_downloaded += 1
                    
                else:
                    pass#print(f"URL not reachable: {url}")
            except Exception as e:
                pass#print(f"Error processing {url}: {e}")
        
        print(f"Downloaded {files_downloaded} of {len(url_list)} files.")  
    # if url_list is a string (single url)   
    elif isinstance(url_list, str):
        try:
            response = requests.get(url_list, stream=True)
            response.raise_for_status()
            filename = os.path.basename(url_list)
            filepath = os.path.join(download_dir, filename)

            with open(filepath, 'wb') as f:
                for chunk in response.iter_content(chunk_size=8192): 
                    if chunk:
                        f.write(chunk)
        except:
            pass



In [5]:
glyphazzn_urls_df = pd.read_csv('https://storage.googleapis.com/magentadata/models/svg_vae/glyphazzn_urls.txt', header=None)
glyphazzn_urls_df.columns = ['id', 'split', 'url']
glyphazzn_urls_df['url'] = glyphazzn_urls_df['url'].str.lower()
glyphazzn_urls_df.drop_duplicates(subset='url', keep='first', inplace=True)
glyphazzn_urls_df['file_format'] = glyphazzn_urls_df['url'].str.split('.').str[-1]
glyphazzn_urls_df = glyphazzn_urls_df[glyphazzn_urls_df['file_format'].isin(['ttf', 'otf'])]
glyphazzn_urls_df['website'] = glyphazzn_urls_df['url'].str.split('/').str[2]
unique_websites = glyphazzn_urls_df['website'].unique().tolist()
website_status_codes = {}
for website in tqdm(unique_websites):
    # We try to get the status code of the website
    try:
        website_status_codes[website] = requests.get('https://' + website).status_code
    except:
        try:
            website_status_codes[website] = requests.get('http://' + website).status_code
        except:
            pass
# We create a new column called status_code
glyphazzn_urls_df['status_code'] = np.nan
# We iterate over the list of website_status_codes and and update the corresponding rows in the dataframe
for website, status_code in website_status_codes.items():
    glyphazzn_urls_df.loc[glyphazzn_urls_df['website'] == website, 'status_code'] = status_code
glyphazzn_urls_df = glyphazzn_urls_df[glyphazzn_urls_df['status_code'] == 200]

urls_list = glyphazzn_urls_df['url'].tolist()
download_dir = os.path.join(os.pardir, 'data', 'raw', 'glyphazzn_urls')


100%|██████████| 23/23 [01:04<00:00,  2.81s/it]


In [ ]:

if not os.path.exists(download_dir):
    os.makedirs(download_dir)
download_fonts(urls_list[2300:], download_dir=download_dir)

In [6]:
def download_fonts_in_parallel(urls, folder):
    if not os.path.exists(folder):
        os.makedirs(folder)

    with ThreadPoolExecutor(max_workers=10) as executor:
        for url in urls:
            executor.submit(download_fonts, url, folder)

This downloads for about 40 minutes about 9500 files (550 MB).

In [7]:
# Now we can download the fonts in parallel
download_dir = os.path.join(os.pardir, 'data', 'raw', 'glyphazzn_urls')
download_fonts_in_parallel(urls_list[4000:], download_dir)

This will give about 3400 filtered files.

In [9]:
df.filter_fonts("../data/raw/glyphazzn_urls", "../data/processed/filtered-fonts")

49it [00:00, 437.19it/s]too much 'glyf' table data: expected 11160, received 11164 bytes
'created' timestamp seems very low; regarding as unix timestamp
'modified' timestamp seems very low; regarding as unix timestamp
'created' timestamp seems very low; regarding as unix timestamp
'modified' timestamp seems very low; regarding as unix timestamp
'created' timestamp seems very low; regarding as unix timestamp
'modified' timestamp seems very low; regarding as unix timestamp
too much 'glyf' table data: expected 74696, received 76964 bytes
'created' timestamp seems very low; regarding as unix timestamp
'modified' timestamp seems very low; regarding as unix timestamp
'created' timestamp out of range; ignoring top bytes
'modified' timestamp out of range; ignoring top bytes
'created' timestamp seems very low; regarding as unix timestamp
'modified' timestamp seems very low; regarding as unix timestamp
too much 'glyf' table data: expected 59746, received 62254 bytes
'created' timestamp seems ver

Removed corrupted file: ../data/raw/glyphazzn_urls\surrogatesmallcaps.ttf
Removed corrupted file: ../data/raw/glyphazzn_urls\swiss_721_black_condensed_bt.ttf


'created' timestamp out of range; ignoring top bytes
'modified' timestamp out of range; ignoring top bytes
'created' timestamp seems very low; regarding as unix timestamp
'modified' timestamp seems very low; regarding as unix timestamp
'created' timestamp seems very low; regarding as unix timestamp
'modified' timestamp seems very low; regarding as unix timestamp
'created' timestamp seems very low; regarding as unix timestamp
'modified' timestamp seems very low; regarding as unix timestamp
6745it [00:30, 152.20it/s]'created' timestamp seems very low; regarding as unix timestamp
'modified' timestamp seems very low; regarding as unix timestamp
'created' timestamp seems very low; regarding as unix timestamp
'modified' timestamp seems very low; regarding as unix timestamp
'created' timestamp seems very low; regarding as unix timestamp
'modified' timestamp seems very low; regarding as unix timestamp
'created' timestamp seems very low; regarding as unix timestamp
'modified' timestamp seems ve

Processed 7786 Fonts. Found 1676 and moved them to ../data/processed/filtered-fonts.


In summation, we have about 3770 filtered font files. Is this enought data? We have to see.